# Deep Learning Bootcamp November 2017, GPU Computing for Data Scientists

<img src="../images/bcamp.png" align="center">

## 31-PyTorch-using-CONV1D-on-one-dimensional-data 

Web: https://www.meetup.com/Tel-Aviv-Deep-Learning-Bootcamp/events/241762893/

Notebooks: <a href="https://github.com/QuantScientist/Data-Science-PyCUDA-GPU"> On GitHub</a>

*Shlomo Kashani*

<img src="../images/pt.jpg" width="35%" align="center">


### Data
- Download from https://numer.ai/leaderboard

# PyTorch Imports


In [6]:
# !pip install pycuda
%reset -f
# %%timeit

import torch
from torch.autograd import Variable
import numpy as np
import pandas
import numpy as np
import pandas as pd
from sklearn import cross_validation
from sklearn import metrics
from sklearn.metrics import roc_auc_score, log_loss, roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
from sklearn import cross_validation
from sklearn import metrics
from sklearn.metrics import roc_auc_score, log_loss, roc_auc_score, roc_curve, auc
from sklearn.cross_validation import StratifiedKFold, ShuffleSplit, cross_val_score, train_test_split
import logging
import numpy
import numpy as np
from __future__ import print_function
from __future__ import division
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import os
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from sklearn.preprocessing import MultiLabelBinarizer
import time
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd
import numpy as np
import scipy
%matplotlib inline
from pylab import rcParams
rcParams['figure.figsize'] = (6, 6)      # setting default size of plots
import tensorflow as tf 
print("tensorflow:" + tf.__version__)
!set "KERAS_BACKEND=tensorflow"
import torch
import sys
print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)
print('__CUDA VERSION')
from subprocess import call
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__Devices')

# !pip install http://download.pytorch.org/whl/cu75/torch-0.2.0.post1-cp27-cp27mu-manylinux1_x86_64.whl
# !pip install torchvision 
# ! pip install cv2
# import cv2

print("OS: ", sys.platform)
print("Python: ", sys.version)
print("PyTorch: ", torch.__version__)
print("Numpy: ", np.__version__)

handler=logging.basicConfig(level=logging.INFO)
lgr = logging.getLogger(__name__)
%matplotlib inline

# !pip install psutil
import psutil
def cpuStats():
        print(sys.version)
        print(psutil.cpu_percent())
        print(psutil.virtual_memory())  # physical memory usage
        pid = os.getpid()
        py = psutil.Process(pid)
        memoryUse = py.memory_info()[0] / 2. ** 30  # memory use in GB...I think
        print('memory GB:', memoryUse)

cpuStats()

# %%timeit
use_cuda = torch.cuda.is_available()
# use_cuda = False

FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
Tensor = FloatTensor

lgr.info("USE CUDA=" + str (use_cuda))


tensorflow:1.2.1
__Python VERSION: 2.7.6 (default, Oct 26 2016, 20:30:19) 
[GCC 4.8.4]
__pyTorch VERSION: 0.2.0_1
__CUDA VERSION
__CUDNN VERSION: 6021
__Number CUDA Devices: 0
__Devices
OS:  linux2
Python:  2.7.6 (default, Oct 26 2016, 20:30:19) 
[GCC 4.8.4]
PyTorch:  0.2.0_1
Numpy:  1.13.1

INFO:__main__:USE CUDA=False



2.7.6 (default, Oct 26 2016, 20:30:19) 
[GCC 4.8.4]
0.5
svmem(total=528393166848, available=416306790400, percent=21.2, used=111776919552, free=388023185408, active=124962492416, inactive=9918504960, buffers=1380556800, cached=27212505088, shared=49983488)
memory GB: 0.198104858398


#  Global params

In [7]:
# NN params
LR = 0.005
MOMENTUM= 0.9

# Data params
TARGET_VAR= 'target'
TOURNAMENT_DATA_CSV = 'numerai_tournament_data.csv'
TRAINING_DATA_CSV = 'numerai_training_data.csv'
BASE_FOLDER = 'numerai/'


# fix seed
seed=17*19
np.random.seed(seed)
torch.manual_seed(seed)
if use_cuda:
    torch.cuda.manual_seed(seed)

#  Train / Validation / Test Split
- Numerai provides a data set that is allready split into train, validation and test sets. 

In [8]:
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from collections import defaultdict

# Train, Validation, Test Split
def loadDataSplit():
    df_train = pd.read_csv(BASE_FOLDER + TRAINING_DATA_CSV)
    # TOURNAMENT_DATA_CSV has both validation and test data provided by NumerAI
    df_test_valid = pd.read_csv(BASE_FOLDER + TOURNAMENT_DATA_CSV)

    answers_1_SINGLE = df_train[TARGET_VAR]
    df_train.drop(TARGET_VAR, axis=1,inplace=True)
    df_train.drop('id', axis=1,inplace=True)
    df_train.drop('era', axis=1,inplace=True)
    df_train.drop('data_type', axis=1,inplace=True)    
            
    df_train.to_csv(BASE_FOLDER + TRAINING_DATA_CSV + 'clean.csv', header=False,  index = False)    
    df_train= pd.read_csv(BASE_FOLDER + TRAINING_DATA_CSV + 'clean.csv', header=None, dtype=np.float32)    
    df_train = pd.concat([df_train, answers_1_SINGLE], axis=1)
    feature_cols = list(df_train.columns[:-1])
    target_col = df_train.columns[-1]
    trainX, trainY = df_train[feature_cols], df_train[target_col]
    
        
    df_validation_set=df_test_valid.loc[df_test_valid['data_type'] == 'validation'] 
    df_validation_set=df_validation_set.copy(deep=True)
    answers_1_SINGLE_validation = df_validation_set[TARGET_VAR]
    df_validation_set.drop(TARGET_VAR, axis=1,inplace=True)    
    df_validation_set.drop('id', axis=1,inplace=True)
    df_validation_set.drop('era', axis=1,inplace=True)
    df_validation_set.drop('data_type', axis=1,inplace=True)
    
    
    df_validation_set.to_csv(BASE_FOLDER + TRAINING_DATA_CSV + '-validation-clean.csv', header=False,  index = False)    
    df_validation_set= pd.read_csv(BASE_FOLDER + TRAINING_DATA_CSV + '-validation-clean.csv', header=None, dtype=np.float32)    
    df_validation_set = pd.concat([df_validation_set, answers_1_SINGLE_validation], axis=1)
    feature_cols = list(df_validation_set.columns[:-1])

    target_col = df_validation_set.columns[-1]
    valX, valY = df_validation_set[feature_cols], df_validation_set[target_col]
                                
    df_test_set = pd.read_csv(BASE_FOLDER + TOURNAMENT_DATA_CSV)
    df_test_set=df_test_set.copy(deep=True)
    df_test_set.drop(TARGET_VAR, axis=1,inplace=True)
    tid_1_SINGLE = df_test_set['id']
    df_test_set.drop('id', axis=1,inplace=True)
    df_test_set.drop('era', axis=1,inplace=True)
    df_test_set.drop('data_type', axis=1,inplace=True)   
            
    feature_cols = list(df_test_set.columns) # must be run here, we dont want the ID    
    df_test_set = pd.concat([tid_1_SINGLE, df_test_set], axis=1)            
    testX = df_test_set[feature_cols].values
        
    return trainX, trainY, valX, valY, testX, df_test_set

In [9]:
trainX, trainY, valX, valY, testX, df_test_set = loadDataSplit()

print (trainX.shape)
print (trainY.shape)
print (valX.shape)
print (valY.shape)
print (testX.shape)
print (df_test_set.shape)

(108405, 21)
(108405,)
(16686, 21)
(16686,)
(45647, 21)
(45647, 22)


In [10]:

# Convert the np arrays into the correct dimention and type
# Note that BCEloss requires Float in X as well as in y
def XnumpyToTensor(x_data_np):
    x_data_np = np.array(x_data_np.values, dtype=np.float32)        
    print(x_data_np.shape)
    print(type(x_data_np))

    if use_cuda:
        lgr.info ("Using the GPU")    
        X_tensor = Variable(torch.from_numpy(x_data_np).cuda()) # Note the conversion for pytorch    
    else:
        lgr.info ("Using the CPU")
        X_tensor = Variable(torch.from_numpy(x_data_np)) # Note the conversion for pytorch
    
    print(type(X_tensor.data)) # should be 'torch.cuda.FloatTensor'            
    print((X_tensor.data.shape)) # torch.Size([108405, 29])
    return X_tensor


# Convert the np arrays into the correct dimention and type
# Note that BCEloss requires Float in X as well as in y
def YnumpyToTensor(y_data_np):    
    y_data_np=y_data_np.reshape((y_data_np.shape[0],1)) # Must be reshaped for PyTorch!
    print(y_data_np.shape)
    print(type(y_data_np))

    if use_cuda:
        lgr.info ("Using the GPU")            
    #     Y = Variable(torch.from_numpy(y_data_np).type(torch.LongTensor).cuda())
        Y_tensor = Variable(torch.from_numpy(y_data_np)).type(torch.FloatTensor).cuda()  # BCEloss requires Float        
    else:
        lgr.info ("Using the CPU")        
    #     Y = Variable(torch.squeeze (torch.from_numpy(y_data_np).type(torch.LongTensor)))  #         
        Y_tensor = Variable(torch.from_numpy(y_data_np)).type(torch.FloatTensor)  # BCEloss requires Float        

    print(type(Y_tensor.data)) # should be 'torch.cuda.FloatTensor'
    print(y_data_np.shape)
    print(type(y_data_np))    
    return Y_tensor

In [13]:
import torch
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


# Dimensions
# Number of features for the input layer
N_FEATURES=trainX.shape[1]
# Number of rows
NUM_ROWS=trainX.shape[0]
MULT_FACTOR=32 # this number has no meaning except for being divisable by 2
Layer1Size=N_FEATURES * MULT_FACTOR
hiddenLayer2Size=int(Layer1Size/4)


X_tensor_train= XnumpyToTensor(trainX) # default order is NBC for a 3d tensor, but we have a 2d tensor
X_shape=X_tensor_train.data.size()

fc1 = torch.nn.Sequential (
   torch.nn.Linear (N_FEATURES, Layer1Size),
   torch.nn.BatchNorm1d(Layer1Size, eps=1e-05, momentum=0.1, affine=True),
   torch.nn.LeakyReLU (0.2),
)

lgr.info  (fc1)
b = fc1(X_tensor_train)
print ('(b.size():' + str (b.size())) # torch.Size([108405, 928])

b = b.view(NUM_ROWS,N_FEATURES,MULT_FACTOR) # torch.Size([108405, 29, 32])
print ('(b.size() after re-shaping:' + str (b.size()))

conv1 = torch.nn.Sequential (
   torch.nn.Conv1d(N_FEATURES, N_FEATURES * MULT_FACTOR, kernel_size=(5,), stride=(1,), padding=(1,)),
   torch.nn.BatchNorm1d(Layer1Size, eps=1e-05, momentum=0.1, affine=True),
   torch.nn.LeakyReLU (0.2),
)

lgr.info (conv1) # 

c = conv1(b) #  Need input of dimension 4 and input.size[1] == 928 but got input to be of shape: [108405 x 29 x 1 x 32]
             # RuntimeError: running_mean should contain 232 elements not 928
lgr.info  (c.size() )

INFO:__main__:Using the CPU
INFO:__main__:Sequential (
  (0): Linear (21 -> 672)
  (1): BatchNorm1d(672, eps=1e-05, momentum=0.1, affine=True)
  (2): LeakyReLU (0.2)
)


(108405, 21)
<type 'numpy.ndarray'>
<class 'torch.FloatTensor'>
torch.Size([108405, 21])


INFO:__main__:Sequential (
  (0): Conv1d(21, 672, kernel_size=(5,), stride=(1,), padding=(1,))
  (1): BatchNorm1d(672, eps=1e-05, momentum=0.1, affine=True)
  (2): LeakyReLU (0.2)
)


(b.size():torch.Size([108405, 672])
(b.size() after re-shaping:torch.Size([108405, 21, 32])


INFO:__main__:torch.Size([108405, 672, 30])


In [26]:
class Net2(nn.Module):    
    def __init__(self, n_feature, n_hidden, n_output,initKernel='uniform'):
        super(Net2, self).__init__()
        self.dis = nn.Sequential(
            nn.Linear(n_feature, n_hidden),
            nn.BatchNorm1d(n_hidden, eps=1e-05, momentum=0.1, affine=True),
            nn.LeakyReLU (0.2),

#             b = b.view(NUM_ROWS,n_feature,MULT_FACTOR), # torch.Size([108405, 29, 32])

            torch.nn.Conv1d(n_feature, n_feature * MULT_FACTOR, kernel_size=(5,), stride=(1,), padding=(1,)),
            torch.nn.BatchNorm1d(n_hidden, eps=1e-05, momentum=0.1, affine=True),
            torch.nn.LeakyReLU (0.2)             
#             nn.Sigmoid()
        )        
    def forward(self, x):
        x = self.dis(x)
        return x                
lgr.info(net)

b = net(X_tensor_train)
# print ('(b.size():' + str (b.size())) # torch.Size([108405, 928])

INFO:__main__:Sequential (
  (0): Linear (21 -> 672)
  (1): BatchNorm1d(672, eps=1e-05, momentum=0.1, affine=True)
  (2): LeakyReLU (0.2)
  (3): Conv1d(21, 672, kernel_size=(5,), stride=(1,), padding=(1,))
  (4): BatchNorm1d(672, eps=1e-05, momentum=0.1, affine=True)
  (5): LeakyReLU (0.2)
)


ValueError: Expected 3D tensor as input, got 2D tensor instead.